## Garbled Tables

It's a distributed computation algorithm between two parties (2PC, or MPC when M=2).

We'll call the parties $P$ and $V$ for no apparent reason. For now.

Let $X$, $Y$ and $Z$ be domains. Let $f: X\times Y \rightarrow Z$

$P$ holds a value $x\in X$ and $V$ holds $y\in Y$. They want to calculate $f(x,y)$ without revealing their input to the other party. This algorithm isn't symmetric, so $P$ will know the values for all he combinations of $x$ and $y$ (let's not focus on efficiency and assume that the table isn't too big).

In [1]:
class P:
    def __init__(self):
        # This is an arbitrary bivariate function that P knows
        self.f = {
            False: {
                False: 1000,
                True: 500
            },
            True: {
                False: 100,
                True: 50
            }
        }

print(P().f)

{False: {False: 1000, True: 500}, True: {False: 100, True: 50}}


Now $P$ wants to share the table with $V$ but not as-is. She will garble it first.

To hide the parameters she will choose two hash functions: $k_X(x)$ and $k_Y(y)$. For security reasons, even if the parameters are of the same type, it's better to choose two different functions.

She then encrypts the results with an encryption function $E(k_1, k_2, z)$ that uses two keys (one for each parameter). $V$ will know the corresponding decrypt function $D(k_1, k_2, z)$.

The garbled table looks like this:

| $x$      | $y$      | $f(x,y)$                  |
| -------- | -------- | ----------------------    |
| $k_X(0)$ | $k_Y(0)$ | $E(k_X(0), k_Y(0), 1000)$ |
| $k_X(0)$ | $k_Y(1)$ | $E(k_X(0), k_Y(1), 500)$  |
| $k_X(1)$ | $k_Y(0)$ | $E(k_X(1), k_Y(0), 100)$  |
| $k_X(1)$ | $k_Y(1)$ | $E(k_X(1), k_Y(1), 50)$   |

$P$ then reorders the keys randomly to get another table, so that $V$ cannot deduct the input values from the indices. She can obtain, for example, the following table:

| $x$      | $y$      | $f(x,y)$                  |
| -------- | -------- | ----------------------    |
| $k_X(1)$ | $k_Y(0)$ | $E(k_X(1), k_Y(0), 100)$  |
| $k_X(1)$ | $k_Y(1)$ | $E(k_X(1), k_Y(1), 50)$   |
| $k_X(0)$ | $k_Y(0)$ | $E(k_X(0), k_Y(0), 1000)$ |
| $k_X(0)$ | $k_Y(1)$ | $E(k_X(0), k_Y(1), 500)$  |

Here, if she appends indexing information to the keys before sending them, she only needs to send the list of values to $V$ for her to use it. An easy way to do this is think about the number as a string and concatenate a character at the end with the corresponding index.

In [9]:
from hashlib import sha256

class P:
    def __init__(self):
        # This is an arbitrary bivariate function that P knows
        self.f = {
            False: {
                False: 1000,
                True: 500
            },
            True: {
                False: 100,
                True: 50
            }
        }

    def offset_x(self, b):
        # X=False goes after than X=True to follow the example
        return 0 if b else 1
    
    def offset_y(self, b):
        # Y=False goes before than Y=True to follow the example
        return 1 if b else 0

    def _hash(self, string):
        return sha256(string.encode()).hexdigest()
    
    def hash_x(self, x):
        return self._hash(str(x))

    def hash_y(self, y):
        # It copies the string twice for hash_x and hash_y to be different
        return self._hash(str(y) + str(y))

    def encrypt(self, x, y, value):
        hashed_x = self.hash_x(x)
        hashed_y = self.hash_y(y)
        
        # This is NOT a safe encryption, but it will suffice for the example
        return int(hashed_x, 16) + int(hashed_y, 16) + value
    
    def garbled_f(self):
        # Complete with encrypted values in the correct order

    def hash_x_with_index(self, x):
        # Complete with the hash with the indexing bit appended at the end

    def hash_y_with_index(self, y):
        # Complete with the hash with the indexing bit appended at the end

p = P()
print("The garbled table values are:")
print(p.garbled_f())

for x in [True, False]:
    print(f"The hashed key for X={x} is\t{p.hash_x_with_index(x)}")

for y in [False, True]:
    print(f"The hashed key for Y={y} is\t{p.hash_y_with_index(y)}")

The garbled table values are:
[105496634696532655740497557342134150169212924173749829350307031622452206611835, 47050336707173276700401674424140008141968049868015890422426145370218942140196, 121735219802917208218106292427731235600517679835230349832487451597460082889445, 63288921813557829178010409509737093573272805529496410904606565345226818417356]
The hashed key for X=True is	3cbc87c7681f34db4617feaa2c8801931bc5e42d8d0f560e756dd4cd92885f180
The hashed key for X=False is	60a33e6cf5151f2d52eddae9685cfa270426aa89d8dbc7dfb854606f1d1a40fe1
The hashed key for Y=False is	ac8072e7867a47e9ec718ceee7ff17e5e1cb2b65636240ba8d068c78d4d74dff0
The hashed key for Y=True is	2b490437a7a6a582967530bd4ccc41d58b6ebd8893e955e26fd8e5a3ec972fda1


The row index is $|Y| \cdot b_x + b_y$. For example the index for $k_X(1),1$ and $k_Y(0),0$ would be $2\cdot 1 + 0 = 2$

(we index from 0 because my CS alter ego won the debate against my maths one)

Then $P$ sends $V$ the last column of the table, that has the values in the correct order.

### What does $V$ use the table for now?

$V$ now has to choose and decrypt the correct row. In order to do that she needs both decryption keys and the index. That is, $(i, k_X(x))$ and $(j, k_Y(y))$.

$P$ can directly send $(i, k_X(x))$ to $V$ because she knows $x$. On the other hand, she would need to know $y$ to determine which $(j, k_Y(y))$ is the one $V$ needs... except that we could find a way for $V$ to ask for a particular value without revealing the key to $P$.

If you solved the adventures in order then you know that this can be achieved with the oblivious transfer protocol.

In [69]:
from hashlib import sha256
from Crypto.PublicKey import RSA
import random

# Copied from OT adventure
def random_noise():
    # N is a really big number
    N = 2**2048
    return random.randrange(N)

class Paula:
    def __init__(self, key, modulus):
        self.x = False
        # This is an arbitrary bivariate function that P knows
        self.f = {
            False: {
                False: 1000,
                True: 500
            },
            True: {
                False: 100,
                True: 50
            }
        }

        # Copied from OT adventure
        self.friend = None
        
        self.key = key
        self.modulus = modulus
        self.xbs = {
            True: random_noise(),
            False: random_noise(),
        }

    def offset_x(self, b):
        # X=False goes after than X=True to follow the example
        return 0 if b else 1
    
    def offset_y(self, b):
        # Y=False goes before than Y=True to follow the example
        return 1 if b else 0

    def _hash(self, string):
        return sha256(string.encode()).hexdigest()
    
    def hash_x(self, x):
        return self._hash(str(x))

    def hash_y(self, y):
        # It copies the string twice for hash_x and hash_y to be different
        return self._hash(str(y) + str(y))

    def encrypt(self, x, y, value):
        hashed_x = self.hash_x(x)
        hashed_y = self.hash_y(y)
        
        # This is NOT a safe encryption, but it will suffice for the example
        return int(hashed_x, 16) + int(hashed_y, 16) + value
    
    def garbled_f(self):
        # Copy from previous snippet

    def hash_x_with_index(self, x):
        # Copy from previous snippet

    def hash_y_with_index(self, y):
        # Copy from previous snippet

    def get_x_key(self):
        return int(self.hash_x_with_index(self.x), 16)

    # Copied from the OT adventure
    def get_xbs(self):
        return self.xbs
    
    def calculate_k_b(self, bit, encrypted_key):
        base = (encrypted_key - self.xbs[bit] + self.modulus) % self.modulus
        return pow(base, self.key, self.modulus)
        
    def calculate_v_b(self, k_b, bit):
        return (k_b + self._f(bit)) % self.modulus
    
    def calculate_encrypted_value(self, bit, encrypted_key):
        k_b = self.calculate_k_b(bit, encrypted_key)
        return self.calculate_v_b(k_b, bit)
    
    def get_f(self, encrypted_key):
        return {
            b: self.calculate_encrypted_value(b, encrypted_key) 
            for b in [True, False]
        }
    
    def _f(self, b):
        # In this case the OT occurs on the hash of the key Y
        return int(self.hash_y_with_index(b), 16)


class Valerie:
    def __init__(self, key, modulus):
        self.y = True

        # Copied from OT adventure
        self.friend = None
        self.key = key
        self.modulus = modulus
        
        self.k = random_noise()

    def get_value_indexed_by(self, garbled_table, x_key_with_index, y_key_with_index):
        x_index = int(x_key_with_index[-1])
        y_index = int(y_key_with_index[-1])
        # Complete returning the corresponding index of the garbled table

    def decrypt(self, encrypted_value, x_key_with_index, y_key_with_index):
        x_key = int(x_key_with_index[:-1], 16)
        y_key = int(y_key_with_index[:-1], 16)

        # This is the inverse of the unsafe encryption from before
        return encrypted_value - x_key - y_key
    
    def calculate_f_with_friend(self):
        garbled_f = self.friend.garbled_f()
        
        # Complete both variables
        #  x_key_with_index = ...
        #  y_key_with_index = ...

        encrypted_value = self.get_value_indexed_by(garbled_f, x_key_with_index, y_key_with_index)
        return self.decrypt(encrypted_value, x_key_with_index, y_key_with_index)

    # Copied from OT adventure and changed a bit
    def get_xb_for_y_value(self):
        return self.friend.get_xbs()[self.y]
    
    def get_correct_encrypted_value(self, hidden_key):
        return self.friend.get_f(hidden_key)[self.y]
    
    def hide_key(self, xb):
        return (xb + pow(self.k, self.key, self.modulus)) % self.modulus
    
    def retrieve_value(self, encrypted_value):
        return (encrypted_value - self.k + self.modulus) % self.modulus
    
    def get_x_key(self):
        return hex(self.friend.get_x_key())
    
    def get_oblivious_y_key(self):
        xb = self.get_xb_for_y_value()
        hidden_key = self.hide_key(xb)
        encrypted_value = self.get_correct_encrypted_value(hidden_key)
        value = self.retrieve_value(encrypted_value)
        return hex(value)

# Copied from OT adventure
class Protocol:
    def __init__(self):
        key_pair = RSA.generate(bits=2048)
        paula_key = key_pair.d
        valerie_key = key_pair.e
        modulus = key_pair.n
        
        self.paula = Paula(paula_key, modulus)
        self.valerie = Valerie(valerie_key, modulus)
        
        self.paula.friend = self.valerie
        self.valerie.friend = self.paula
        
protocol = Protocol()
value = protocol.valerie.calculate_f_with_friend()
print(f"The calculated value is {value}")

The calculated value is 500
